In [ ]:
#install it only once.
pip install cmake
pip install dlib-19.24.1-cp311-cp311-win_amd64.whl

In [1]:
#Final Drowsiness Model.
from flask import Flask, request, jsonify
import base64
import io
from PIL import Image, ExifTags
from keras.models import load_model
import cv2
import numpy as np
import dlib

app = Flask(__name__)

# function to load the models
def load_model_fn(path):
    model = load_model(path)
    print('Model loaded!')
    return model

drowsiness_model = load_model_fn("drowsinessmodel.h5")

def exif_transpose(img):
    try:
        for orientation in ExifTags.TAGS.keys():
            if ExifTags.TAGS[orientation]=='Orientation':
                break
        e = img._getexif()
        if e is not None:
            exif=dict(e.items())
            if orientation in exif:
                if exif[orientation] == 3:
                    img = img.transpose(Image.ROTATE_180)
                elif exif[orientation] == 6:
                    img = img.transpose(Image.ROTATE_270)
                elif exif[orientation] == 8:
                    img = img.transpose(Image.ROTATE_90)
    except Exception as e:
        print(f"Error in exif_transpose: {e}")
    return img


predictor_path = "shape_predictor_68_face_landmarks.dat"
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(predictor_path)

def crop_mouth(img, Target_Size=(224, 224)):
    
#     img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) 

    # Ask the detector to find the bounding boxes of each face. The 1 in the
    # second argument indicates that we should upsample the image 1 time. This
    # will make everything bigger and allow us to detect more faces.
    dets = detector(img, 1)
    if len(dets) > 0:
        for k, d in enumerate(dets):
            # Get the landmarks/parts for the face in box d.
            shape = predictor(img, d)
            # The next lines of code just get the coordinates for the mouth
            # and crop the mouth from the image.This part can probably be optimised
            # by taking only the outer most points.
            xmouthpoints = [shape.part(x).x for x in range(48,67)]
            ymouthpoints = [shape.part(x).y for x in range(48,67)]
            maxx = max(xmouthpoints)
            minx = min(xmouthpoints)
            maxy = max(ymouthpoints)
            miny = min(ymouthpoints) 

            # to show the mouth properly pad both sides
            pad = 10

            crop_image = img[miny-pad:maxy+pad,minx-pad:maxx+pad]
            RGB_image = cv2.cvtColor(crop_image, cv2.COLOR_BGR2RGB)
            resized_image = cv2.resize(RGB_image,Target_Size)
            return resized_image
    else:
        return None
    
def crop_eye(img, Target_Size=(224, 224)):
    
#     img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) 

    # Ask the detector to find the bounding boxes of each face. The 1 in the
    # second argument indicates that we should upsample the image 1 time. This
    # will make everything bigger and allow us to detect more faces.
    dets = detector(img, 1)
    if len(dets) > 0:
        for k, d in enumerate(dets):
            # Get the landmarks/parts for the face in box d.
            shape = predictor(img, d)
            # The next lines of code just get the coordinates for the mouth
            # and crop the eye from the image.This part can probably be optimised
            # by taking only the outer most points.
            xmouthpoints = [shape.part(18).x, shape.part(19).x, shape.part(20).x, shape.part(21).x, shape.part(36).x, shape.part(37).x, shape.part(38).x, shape.part(39).x, shape.part(40).x,shape.part(41).x]
            ymouthpoints = [shape.part(18).y, shape.part(19).y, shape.part(20).y, shape.part(21).y, shape.part(36).y, shape.part(37).y, shape.part(38).y, shape.part(39).y, shape.part(40).y,shape.part(41).y]
            maxx = max(xmouthpoints)
            minx = min(xmouthpoints)
            maxy = max(ymouthpoints)
            miny = min(ymouthpoints) 

            # to show the mouth properly pad both sides
            pad = 10

            crop_image = img[miny-pad:maxy+pad,minx-pad:maxx+pad]
            RGB_image = cv2.cvtColor(crop_image, cv2.COLOR_BGR2RGB)
            resized_image = cv2.resize(RGB_image,Target_Size)
            return resized_image
    else:
        return None

@app.route('/drowsiness', methods=['POST'])
def predict_drowsiness():
    message = request.get_json(force=True)
    encoded = message['image']
    decoded = base64.b64decode(encoded)
    image_received = Image.open(io.BytesIO(decoded))

    # Correct the orientation
    image_received = exif_transpose(image_received)
    image_arr = np.array(image_received)
    mapping_list = ["yawn", "no_yawn", "Closed", "Open"]
    try:
        #crop eyes
        eye_right = crop_eye(image_arr)
        # Reshape and normalize the image
        img_batch_right_eye = np.expand_dims(eye_right, axis=0)
        normalized_eye = img_batch_right_eye / 255
        # Make prediction for right eye
        prediction_eye_right = drowsiness_model.predict(normalized_eye).tolist()
        index_of_eye_right = np.argmax(prediction_eye_right)
        largest_eye = max(prediction_eye_right[0])
        if largest_eye > 0.50:
            result_right = mapping_list[index_of_eye_right]
        else:
            result_right = 'Weak Prediction'
        
    except:
        result_right = "Couldn't Crop the Eye"
        
    try:
        #crop mouth and resize
        mouth_image = crop_mouth(image_arr)
        img_batch_mouth = np.expand_dims(mouth_image, axis=0)
        normalized_mouth = img_batch_mouth / 255
        #make mouth prediction
        prediction_mouth = drowsiness_model.predict(normalized_mouth).tolist()
        index_of_mouth = np.argmax(prediction_mouth[0])
        largest_mouth = max(prediction_mouth[0])
        if largest_mouth > 0.50:
            result_mouth = mapping_list[index_of_mouth]
        else:
            result_mouth = 'Weak Prediction'
        
    except:
        result_mouth = "Couldn't Crop the Mouth"
    
    # Initial value for final_result
    final_result = ""
    
    if result_mouth=="yawn" or result_right == "Closed":
        final_result = "Drowsy"
    else:
        final_result = 'Awake'
    
    return jsonify(final_result)

if __name__ == "__main__":
    app.run(host='0.0.0.0', port=5000)

Model loaded!
 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.1.11:5000
Press CTRL+C to quit


1/1 [==============================] - 0s 57ms/step


192.168.1.17 - - [23/Feb/2024 19:53:15] "POST /drowsiness HTTP/1.1" 200 -


1/1 [==============================] - 0s 53ms/step


...

192.168.1.17 - - [23/Feb/2024 20:01:22] "POST /drowsiness HTTP/1.1" 200 -


1/1 [==============================] - 0s 55ms/step


192.168.1.17 - - [23/Feb/2024 20:01:24] "POST /drowsiness HTTP/1.1" 200 -


1/1 [==============================] - 0s 60ms/step


192.168.1.17 - - [23/Feb/2024 20:01:25] "POST /drowsiness HTTP/1.1" 200 -
192.168.1.17 - - [23/Feb/2024 20:01:26] "POST /drowsiness HTTP/1.1" 200 -
